In [1]:
import os
import datetime
import random
import os.path

import pandas as pd
import numpy as np

#生成するデータファイルの格納先のpathを指定してください
FILE_PATH = '' 
#店舗数
NUM_SHOP = 50
#商品数
NUM_GOODS = 200

In [2]:
pd.__version__

'0.22.0'

In [3]:
#日付作成
start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2016, 6, 30)

date_list = []
while start < end:
    date_list.append(start.strftime('%Y/%m/%d'))
    start +=  datetime.timedelta(days=1)
    
NUM_SHOP_goods = NUM_SHOP + NUM_GOODS


In [4]:
#ID(店鋪&商品)取得
random_id = random.sample(range(NUM_SHOP_goods), NUM_SHOP_goods)

#店鋪idと商品idが連動しないようにした
id_shop = random_id[:NUM_SHOP]
id_goods = random_id[NUM_SHOP:]

In [5]:
shop_list_data_day = []
goods_list_data_day = []

for shop in id_shop:
    shopid_data = [shop]*len(id_goods)
    shop_list_data_day += shopid_data
    goods_list_data_day += id_goods

# 1日あたり生成されるデータ数
length_shop_goods_par_day = len(goods_list_data_day)


day_list_test_data = []
shop_list_test_data = []
goods_list_test_data = []


for day in date_list:
    day_test_data = [day]*length_shop_goods_par_day
    day_list_test_data += day_test_data
    shop_list_test_data += shop_list_data_day
    goods_list_test_data += goods_list_data_day
    
    
    

# 売上高と販売数量が一様乱数の場合

In [6]:
#1の答え
#売上高と販売数量は0~1000の一様乱数にした

main_df = pd.DataFrame({
                      'shop_id' : shop_list_test_data,
                      'goods_id':goods_list_test_data,
                      'sale':np.random.rand(len(day_list_test_data)) * 1000,
                      'sales_quantity':np.random.rand(len(day_list_test_data)) * 1000
                     }, index=pd.to_datetime(day_list_test_data))
#保存
main_df.to_csv( FILE_PATH)

In [7]:
#500MB以上あるかチェック

getsize_df=os.path.getsize(FILE_PATH )*9.536743164e-7


def check_df_size(_getsize_df):
    #500MB以上あるのか判定
    if _getsize_df>500:
        print("作成されたcsvは {:.2f}MBなので正常終了".format(_getsize_df))
        return True
    else:
        print("作成されたcsvは {:.2f}MBなので、あと {:.2f}MB足りません".format(_getsize_df,500-_getsize_df))
        raise Exception('capacity lack')
        
check_df_size(getsize_df)

作成されたcsvは 662.16MBなので正常終了


True

In [8]:
df_sum_month_group = main_df.groupby(pd.Grouper(freq='MS'))

df_month_shop_list =[]
df_month_goods_list =[]

#monthには毎月の1日が格納されている
for month in df_sum_month_group.sum().index[:]:
    df_month = df_sum_month_group.get_group(month)
    
    #shop_id,goodsごとのsumをとる
    df_month_shop_group = df_month.groupby('shop_id').sum()
    df_month_goods_group = df_month.groupby('goods_id').sum()
    
    df_month_shop_group['date'] = month
    df_month_goods_group['date'] = month
    
    df_month_shop_list.append(df_month_shop_group.loc[:, ['date','sale']])
    df_month_goods_list.append(df_month_goods_group.loc[:, ['date','sale']])
    
#2.1と2.２の回答
df_month_shop = pd.concat(df_month_shop_list).reset_index().loc[:,['date','shop_id','sale']]
df_month_goods = pd.concat(df_month_goods_list).reset_index().loc[:,['date','goods_id','sale']]


In [9]:
#2.3の回答

#有効数字10桁に変更
pd.options.display.precision = 10
def get_sale_division(_x):
    sale,sales_quantity = _x
    #有効数字10桁
    return float(format(sale/sales_quantity, '.10f'))


df_day_group = main_df.groupby(pd.Grouper(freq='D'))

df_goods_mean_list = []
for day in df_day_group.sum().index[:]:
    
    df_day_division_list = []
    df_day = df_day_group.get_group(day)
    df_day_shop_group = df_day.groupby('shop_id')
    for shop_id in id_shop[:]:
        df_day_shop = df_day_shop_group.get_group(shop_id).copy()
        df_day_shop['sale/sale_quantity'] = df_day_shop.loc[:,['sale','sales_quantity']].apply(get_sale_division,axis=1)
        df_day_division_list.append(df_day_shop)
    df_shop_id_day_mean = pd.concat(df_day_division_list).groupby('goods_id').mean()
    df_shop_id_day_mean['day'] = day
    df_goods_mean_list.append(df_shop_id_day_mean)
    

df_goods_mean = pd.concat(df_goods_mean_list).reset_index().loc[:, ['day','goods_id','sale/sale_quantity']]


# 解答(5行まで出力)

In [10]:
#1
main_df.head()

,goods_id,sale,sales_quantity,shop_id
2013-01-01,107,9.5534230623,355.6812675999,66
2013-01-01,203,733.1375249226,204.7802622928,66
2013-01-01,6,36.8355920021,686.1620768920,66
2013-01-01,183,580.1201881010,200.6342933389,66
2013-01-01,36,217.0481734970,582.4778378337,66


In [11]:
#2.1
df_month_shop.head()

,date,shop_id,sale
0,2013-01-01,3,3.1018225243e+06
1,2013-01-01,7,3.1052421788e+06
2,2013-01-01,17,3.0760369845e+06
3,2013-01-01,20,3.0972320584e+06
4,2013-01-01,25,3.0983841108e+06


In [12]:
#2.2
df_month_goods.head()

,date,goods_id,sale
0,2013-01-01,0,774825.2462648593
1,2013-01-01,1,752302.6217068972
2,2013-01-01,2,761064.9996454796
3,2013-01-01,4,774542.9842884388
4,2013-01-01,5,766257.9733379297


In [13]:
#2.3
df_goods_mean.head()

,day,goods_id,sale/sale_quantity
0,2013-01-01,0,1.2878012114
1,2013-01-01,1,1.5515892244
2,2013-01-01,2,1.5389206563
3,2013-01-01,4,9.5425605030
4,2013-01-01,5,2.7550923200
